In [1]:
import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.metrics import Accuracy

In [2]:
from __future__ import division, print_function, absolute_import

from sklearn.model_selection import train_test_split
from skimage import color, io
from scipy.misc import imresize
import numpy as np
import os
from glob import glob

import cv2
import numpy as np
import os         
from random import shuffle 
from tqdm import tqdm      
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
# First access the training data set


In [4]:
train_data = np.load('train_data.npy')
test_data = np.load('test_data.npy')

In [5]:
train_data.dtype

dtype('O')

In [6]:
IMG_SIZE = 64
LR = 1e-3

In [7]:
X_train = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = [i[1] for i in train_data]

In [8]:
X, X_test, Y, Y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [9]:
X_test.dtype

dtype('uint8')

In [10]:
X_test = X_test.astype(np.float64)
X = X.astype(np.float64)
X.dtype

dtype('float64')

In [11]:
####### Image normalistation
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping & rotating images
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [12]:

###################################
# Define network architecture

# Input is a 32x32 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, 64, 64, 1],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)

# 1: Convolution layer with 32 filters, each 3x3x3
conv_1 = conv_2d(network, 32, 1, activation='relu', name='conv_1')

# 2: Max pooling layer
network = max_pool_2d(conv_1, 2)

# 3: Convolution layer with 64 filters
conv_2 = conv_2d(network, 64, 1, activation='relu', name='conv_2')

# 4: Convolution layer with 64 filters
conv_3 = conv_2d(conv_2, 64, 1, activation='relu', name='conv_3')

# 5: Max pooling layer
network = max_pool_2d(conv_3, 2)

# 6: Fully-connected 512 node layer
network = fully_connected(network, 512, activation='relu')

# 7: Dropout layer to combat overfitting
network = dropout(network, 0.5)

# 8: Fully-connected layer with two outputs
network = fully_connected(network, 2, activation='softmax')

# Configure how the network will be trained
acc = Accuracy(name="Accuracy")
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0005, metric=acc)





# convnet = conv_2d(convnet, 32, 5, activation='relu')
# convnet = max_pool_2d(convnet, 5)

# convnet = conv_2d(convnet, 64, 5, activation='relu')
# convnet = max_pool_2d(convnet, 5)

# convnet = conv_2d(convnet, 128, 5, activation='relu')
# convnet = max_pool_2d(convnet, 5)

# convnet = conv_2d(convnet, 64, 5, activation='relu')
# convnet = max_pool_2d(convnet, 5)

# convnet = conv_2d(convnet, 32, 5, activation='relu')
# convnet = max_pool_2d(convnet, 5)

# convnet = fully_connected(convnet, 1024, activation='relu')
# convnet = dropout(convnet, 0.8)

# convnet = fully_connected(convnet, 2, activation='softmax')
# convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

In [13]:

# Wrap the network in a model object
model = tflearn.DNN(network, checkpoint_path='tsubasa.tflearn', max_checkpoints = 3,
                    tensorboard_verbose = 3, tensorboard_dir='tmp/new/')

###################################
# Train model for 100 epochs
###################################


In [ ]:
X.shape

In [ ]:
model.fit(X, Y, validation_set=(X_test, Y_test), batch_size=300,n_epoch=100, run_id='tsubasa', show_metric=True)

model.save('tsubasa_final.tflearn')

In [ ]:
model.fit(X,Y, n_epoch=10, 
          validation_set=(X_test, Y_test), 
          snapshot_step=500, show_metric=True, run_id='tsubasa')
model.save('tsubasa_final.tflearn')

In [14]:
model.load('tsubasa_final.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\nile6\tsubasa_final.tflearn


In [ ]:
def create_test_data():
    testing_data = []
    for img in tqdm(os.listdir('C:/Users/nile6/lol')):
        path = os.path.join('C:/Users/nile6/lol',img)
        img_num = img.split('.')[0]
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img_data), img_num])
        
    shuffle(testing_data)
    np.save('test_lol.npy', testing_data)
    return testing_data

In [ ]:
test_data = create_test_data()

In [ ]:
k = 0
for i in test_data:
    img_data, img_num = i
    img_data = img_data.astype(np.float64)
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    data = data.astype(np.float64)
    prediction = model.predict([data])[0]
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111)
    ax.imshow(img_data, cmap="gray")
    k = k+1
    print(f"cat: {prediction[0]}, dog: {prediction[1]} {k}")
    plt.show()

In [ ]:
d = test_data[0]
img_data, img_num = d 

img_data = img_data.astype(np.float64)

In [ ]:
img_data = img_data.astype(np.float64)

data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
data = data.astype(np.float64)
prediction = model.predict([data])[0]

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.imshow(img_data, cmap="gray")
print(f"cat: {prediction[0]}, dog: {prediction[1]}")
# ax.imshow(img_data, cmap="gray")
plt.show()

In [ ]:
plt.close('all')

In [ ]:
# from IPython.display import Image
Image("C:/Users/nile6/lol/25.jpg")

# To open in default app image
# path = 'C:/Users/nile6/lol/25.jpg'
# from PIL import Image


# image = Image.open(path)
# image.show()

In [54]:
import cv2
import numpy as np

# Playing video from file:
# cap = cv2.VideoCapture('vtest.avi')
# Capturing video from webcam:
cap = cv2.VideoCapture('plz.mp4')

currentFrame = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Handles the mirroring of the current frame
    frame = cv2.flip(frame,1)

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Saves image of the current frame in jpg file
    # name = 'frame' + str(currentFrame) + '.jpg'
    # cv2.imwrite(name, frame)

    # Display the resulting frame
   
    img_data = cv2.resize(gray, (IMG_SIZE, IMG_SIZE))
    img_data = img_data.astype(np.float64)
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    data = data.astype(np.float64)
    prediction = model.predict([data])[0]
    print("cat: {:.2f}, dog: {:.2f}".format(prediction[0],prediction[1]))
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,str("cat: {:.2f}, dog: {:.2f}".format(prediction[0],prediction[1])),(5,80), font, 1,(255,255,255),2,cv2.LINE_AA)
#     cv2.putText(frame,"Hello World!!!", (55,150), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # To stop duplicate images
    currentFrame += 1
   # When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.61, dog: 0.39
cat: 0.61, dog: 0.39
cat: 0.61, dog: 0.39
cat: 0.62, dog: 0.38
cat: 0.62, dog: 0.38
cat: 0.57, dog: 0.43
cat: 0.60, dog: 0.40
cat: 0.61, dog: 0.39
cat: 0.61, dog: 0.39
cat: 0.58, dog: 0.42
cat: 0.60, dog: 0.40
cat: 0.62, dog: 0.38
cat: 0.62, dog: 0.38
cat: 0.62, dog: 0.38
cat: 0.61, dog: 0.39
cat: 0.61, dog: 0.39
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.60, dog: 0.40
cat: 0.59, dog: 0.41
cat: 0.56, dog: 0.44
cat: 0.56, dog: 0.44
cat: 0.55, dog: 0.45
cat: 0.55, dog: 0.45
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.60, dog: 0.40
cat: 0.59, dog: 0.41
cat: 0.60, dog: 0.40
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.61, dog: 0.39
cat: 0.61, dog: 0.39
cat: 0.57, dog: 0.43
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.55, dog: 0.45
cat: 0.56, dog: 0.44
cat: 0.56, dog: 0.44
cat: 0.55, dog: 0.45
cat: 0.55, dog: 0.45
cat: 0.59, dog: 0.41
cat: 0.58, do

cat: 0.57, dog: 0.43
cat: 0.57, dog: 0.43
cat: 0.57, dog: 0.43
cat: 0.56, dog: 0.44
cat: 0.57, dog: 0.43
cat: 0.57, dog: 0.43
cat: 0.59, dog: 0.41
cat: 0.58, dog: 0.42
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.60, dog: 0.40
cat: 0.60, dog: 0.40
cat: 0.60, dog: 0.40
cat: 0.58, dog: 0.42
cat: 0.57, dog: 0.43
cat: 0.57, dog: 0.43
cat: 0.58, dog: 0.42
cat: 0.56, dog: 0.44
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.59, dog: 0.41
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.59, dog: 0.41
cat: 0.58, dog: 0.42
cat: 0.57, dog: 0.43
cat: 0.57, dog: 0.43
cat: 0.57, dog: 0.43
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.58, dog: 0.42
cat: 0.44, dog: 0.56
cat: 0.45, do

cat: 0.58, dog: 0.42
cat: 0.62, dog: 0.38
cat: 0.62, dog: 0.38
cat: 0.62, dog: 0.38
cat: 0.56, dog: 0.44
cat: 0.55, dog: 0.45
cat: 0.56, dog: 0.44
cat: 0.55, dog: 0.45
cat: 0.55, dog: 0.45
cat: 0.55, dog: 0.45
cat: 0.50, dog: 0.50
cat: 0.52, dog: 0.48
cat: 0.53, dog: 0.47
cat: 0.53, dog: 0.47
cat: 0.53, dog: 0.47
cat: 0.53, dog: 0.47
cat: 0.53, dog: 0.47
cat: 0.53, dog: 0.47
cat: 0.53, dog: 0.47
cat: 0.55, dog: 0.45
cat: 0.55, dog: 0.45
cat: 0.56, dog: 0.44
cat: 0.48, dog: 0.52
cat: 0.49, dog: 0.51
cat: 0.49, dog: 0.51
cat: 0.49, dog: 0.51
cat: 0.49, dog: 0.51
cat: 0.49, dog: 0.51
cat: 0.51, dog: 0.49
cat: 0.48, dog: 0.52
cat: 0.48, dog: 0.52
cat: 0.52, dog: 0.48
cat: 0.51, dog: 0.49
cat: 0.51, dog: 0.49
cat: 0.53, dog: 0.47
cat: 0.51, dog: 0.49
cat: 0.50, dog: 0.50
cat: 0.52, dog: 0.48
cat: 0.53, dog: 0.47
cat: 0.52, dog: 0.48
cat: 0.51, dog: 0.49
cat: 0.52, dog: 0.48
cat: 0.52, dog: 0.48
cat: 0.48, dog: 0.52
cat: 0.46, dog: 0.54
cat: 0.46, dog: 0.54
cat: 0.50, dog: 0.50
cat: 0.57, do

cat: 0.46, dog: 0.54
cat: 0.52, dog: 0.48
cat: 0.52, dog: 0.48
cat: 0.48, dog: 0.52
cat: 0.46, dog: 0.54
cat: 0.44, dog: 0.56
cat: 0.45, dog: 0.55
cat: 0.46, dog: 0.54
cat: 0.44, dog: 0.56
cat: 0.47, dog: 0.53
cat: 0.49, dog: 0.51
cat: 0.47, dog: 0.53
cat: 0.48, dog: 0.52
cat: 0.48, dog: 0.52
cat: 0.49, dog: 0.51
cat: 0.47, dog: 0.53
cat: 0.46, dog: 0.54
cat: 0.43, dog: 0.57
cat: 0.46, dog: 0.54
cat: 0.48, dog: 0.52
cat: 0.50, dog: 0.50
cat: 0.51, dog: 0.49
cat: 0.55, dog: 0.45
cat: 0.53, dog: 0.47
cat: 0.47, dog: 0.53
cat: 0.49, dog: 0.51
cat: 0.53, dog: 0.47
cat: 0.53, dog: 0.47
cat: 0.51, dog: 0.49
cat: 0.47, dog: 0.53
cat: 0.46, dog: 0.54
cat: 0.47, dog: 0.53
cat: 0.46, dog: 0.54
cat: 0.45, dog: 0.55
cat: 0.48, dog: 0.52
cat: 0.49, dog: 0.51
cat: 0.48, dog: 0.52
cat: 0.48, dog: 0.52
cat: 0.47, dog: 0.53
cat: 0.46, dog: 0.54
cat: 0.47, dog: 0.53
cat: 0.50, dog: 0.50
cat: 0.49, dog: 0.51
cat: 0.49, dog: 0.51
cat: 0.54, dog: 0.46
cat: 0.55, dog: 0.45
cat: 0.54, dog: 0.46
cat: 0.58, do

cat: 0.45, dog: 0.55
cat: 0.45, dog: 0.55
cat: 0.44, dog: 0.56
cat: 0.41, dog: 0.59
cat: 0.35, dog: 0.65
cat: 0.40, dog: 0.60
cat: 0.35, dog: 0.65
cat: 0.41, dog: 0.59
cat: 0.41, dog: 0.59
cat: 0.40, dog: 0.60
cat: 0.44, dog: 0.56
cat: 0.41, dog: 0.59
cat: 0.41, dog: 0.59
cat: 0.42, dog: 0.58
cat: 0.43, dog: 0.57
cat: 0.44, dog: 0.56
cat: 0.42, dog: 0.58
cat: 0.38, dog: 0.62
cat: 0.40, dog: 0.60
cat: 0.40, dog: 0.60
cat: 0.37, dog: 0.63
cat: 0.40, dog: 0.60
cat: 0.40, dog: 0.60
cat: 0.43, dog: 0.57
cat: 0.37, dog: 0.63
cat: 0.37, dog: 0.63
cat: 0.37, dog: 0.63
cat: 0.43, dog: 0.57
cat: 0.42, dog: 0.58
cat: 0.41, dog: 0.59
cat: 0.42, dog: 0.58
cat: 0.39, dog: 0.61
cat: 0.39, dog: 0.61
cat: 0.39, dog: 0.61
cat: 0.39, dog: 0.61
cat: 0.42, dog: 0.58
cat: 0.40, dog: 0.60
cat: 0.38, dog: 0.62
cat: 0.40, dog: 0.60
cat: 0.41, dog: 0.59
cat: 0.40, dog: 0.60
cat: 0.37, dog: 0.63
cat: 0.40, dog: 0.60
cat: 0.43, dog: 0.57
cat: 0.43, dog: 0.57
cat: 0.45, dog: 0.55
cat: 0.42, dog: 0.58
cat: 0.41, do

cat: 0.84, dog: 0.16
cat: 0.90, dog: 0.10
cat: 0.86, dog: 0.14
cat: 0.90, dog: 0.10
cat: 0.88, dog: 0.12
cat: 0.87, dog: 0.13
cat: 0.88, dog: 0.12
cat: 0.85, dog: 0.15
cat: 0.76, dog: 0.24
cat: 0.77, dog: 0.23
cat: 0.73, dog: 0.27
cat: 0.72, dog: 0.28
cat: 0.68, dog: 0.32
cat: 0.64, dog: 0.36
cat: 0.72, dog: 0.28
cat: 0.82, dog: 0.18
cat: 0.84, dog: 0.16
cat: 0.82, dog: 0.18
cat: 0.78, dog: 0.22
cat: 0.80, dog: 0.20
cat: 0.78, dog: 0.22
cat: 0.81, dog: 0.19
cat: 0.79, dog: 0.21
cat: 0.81, dog: 0.19
cat: 0.90, dog: 0.10
cat: 0.88, dog: 0.12
cat: 0.81, dog: 0.19
cat: 0.75, dog: 0.25
cat: 0.70, dog: 0.30
cat: 0.76, dog: 0.24
cat: 0.82, dog: 0.18
cat: 0.81, dog: 0.19
cat: 0.81, dog: 0.19
cat: 0.74, dog: 0.26
cat: 0.79, dog: 0.21
cat: 0.86, dog: 0.14
cat: 0.81, dog: 0.19
cat: 0.83, dog: 0.17
cat: 0.81, dog: 0.19
cat: 0.80, dog: 0.20
cat: 0.81, dog: 0.19
cat: 0.80, dog: 0.20
cat: 0.78, dog: 0.22
cat: 0.82, dog: 0.18
cat: 0.84, dog: 0.16
cat: 0.86, dog: 0.14
cat: 0.87, dog: 0.13
cat: 0.84, do

cat: 0.55, dog: 0.45
cat: 0.56, dog: 0.44
cat: 0.58, dog: 0.42
cat: 0.57, dog: 0.43
cat: 0.56, dog: 0.44
cat: 0.56, dog: 0.44
cat: 0.56, dog: 0.44


In [25]:
def create_test_data(frame):
    testing_data = []
    img_data = cv2.imread(frame, cv2.IMREAD_GRAYSCALE)
    img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
    testing_data.append([np.array(img_data)])
    np.save('test_lol.npy', testing_data)
    return testing_data

In [ ]:
import numpy as np
import cv2

img = cv2.imread('cat.jpg',cv2.IMREAD_COLOR)
cv2.line(img,(0,0),(150,150),(255,255,255),15)

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [52]:
prediction[0] = 3.454546546565465465464256
print("cat: {:.2f}, dog: {:.2f}".format(prediction[0],prediction[1]))

cat: 3.45, dog: 0.41
